In [57]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


## loading dataset

In [1]:
#load the iris dataset
dataset = pd.read_csv('iris.csv')  

X = dataset.iloc[:,1:-1].values
y = dataset.iloc[:,-1].values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

print(X)
print(y)

NameError: name 'pd' is not defined

## split first 40 data points of each class into training set and remaining 10 points into test set

In [59]:
#Separate data into different classes
setosa_indices = np.where(y == 0)[0]
versicolor_indices = np.where(y == 1)[0]
virginica_indices = np.where(y == 2)[0]


X_train = np.concatenate((X[setosa_indices[:40]], X[versicolor_indices[:40]], X[virginica_indices[:40]]), axis = 0)
y_train = np.concatenate((y[setosa_indices[:40]], y[versicolor_indices[:40]], y[virginica_indices[:40]]), axis = 0)
X_test = np.concatenate((X[setosa_indices[40:]], X[versicolor_indices[40:]], X[virginica_indices[40:]]), axis = 0)
y_test = np.concatenate((y[setosa_indices[40:]], y[versicolor_indices[40:]], y[virginica_indices[40:]]), axis = 0)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)



(120, 4)
(120,)
(30, 4)
(30,)


## calculating mean vectors and inverse covariance matrix of train samples

In [60]:
mean_vector_setosa = np.mean(X_train[y_train == 0], axis=0)
print("mean_vector of setosa:\n",mean_vector_setosa)
cov_matrix_setosa = np.cov(X_train[y_train == 0], rowvar=False)
print("covariance_mat of setosa:\n",cov_matrix_setosa)
inv_cov_matrix_setosa = np.linalg.inv(cov_matrix_setosa)
print("inv_cov of setosa:\n",inv_cov_matrix_setosa)

print("\n\n\n")

mean_vector_versicolor= np.mean(X_train[y_train == 1], axis=0)
print("mean_vector of versicolor:\n",mean_vector_versicolor)
cov_matrix_versicolor = np.cov(X_train[y_train == 1], rowvar=False)
print("covariance_mat of versicolor:\n",cov_matrix_versicolor)
inv_cov_matrix_versicolor = np.linalg.inv(cov_matrix_versicolor)
print("inv_cov of versicolor:\n",inv_cov_matrix_versicolor)

print("\n\n\n")

mean_vector_virginica = np.mean(X_train[y_train == 2], axis=0)
print("mean_vector of virginica:\n",mean_vector_virginica)
cov_matrix_virginica = np.cov(X_train[y_train == 2], rowvar=False)
print("covariance_mat of virginica:\n",cov_matrix_virginica)
inv_cov_matrix_virginica = np.linalg.inv(cov_matrix_virginica)
print("inv_cov of virginica:\n",inv_cov_matrix_virginica)


mean_vector of setosa:
 [5.0375 3.44   1.4625 0.2325]
covariance_mat of setosa:
 [[0.13112179 0.09897436 0.01298077 0.01362179]
 [0.09897436 0.13271795 0.00205128 0.0145641 ]
 [0.01298077 0.00205128 0.02958333 0.00458333]
 [0.01362179 0.0145641  0.00458333 0.00994231]]
inv_cov of setosa:
 [[ 18.96620997 -13.74680415  -6.95996886  -2.63963858]
 [-13.74680415  19.00598614   6.57939373 -12.03991503]
 [ -6.95996886   6.57939373  39.21733422 -18.18107202]
 [ -2.63963858 -12.03991503 -18.18107202 130.21494869]]




mean_vector of versicolor:
 [6.01   2.78   4.3175 1.35  ]
covariance_mat of versicolor:
 [[0.27374359 0.08661538 0.17212821 0.05230769]
 [0.08661538 0.11087179 0.08087179 0.04538462]
 [0.17212821 0.08087179 0.20353205 0.07371795]
 [0.05230769 0.04538462 0.07371795 0.04307692]]
inv_cov of versicolor:
 [[  9.13333334  -4.07640785  -9.58250752   9.60294701]
 [ -4.07640785  17.71179608   3.61868629 -19.90341079]
 [ -9.58250752   3.61868629  23.00463124 -31.54468702]
 [  9.60294701 -19

#  multivariate_normal_density formula code

In [53]:
def multivariate_normal_density(x, mean_vector, inv_cov):
    p = x.shape[0]
    det_inv_cov = np.linalg.det(inv_cov)
    exponent = -0.5 * np.dot(np.dot((x - mean_vector).T, inv_cov), (x - mean_vector))
    coef = 1 / ((2 * np.pi) ** (p / 2) * np.sqrt(det_inv_cov))
    return coef * np.exp(exponent)


###  finding multivariate_normal_density for each sample in test set for each class and finding the predicted class for each sample in test set

In [54]:
# finding multivariate_normal_density for each sample in test set for each class
setosa_density = np.zeros(X_test.shape[0])
versicolor_density = np.zeros(X_test.shape[0])
virginica_density = np.zeros(X_test.shape[0])
for i in range(X_test.shape[0]):
    setosa_density[i] = multivariate_normal_density(X_test[i], mean_vector_setosa, inv_cov_matrix_setosa)

    versicolor_density[i] = multivariate_normal_density(X_test[i], mean_vector_versicolor, inv_cov_matrix_versicolor)
    
    virginica_density[i] = multivariate_normal_density(X_test[i], mean_vector_virginica, inv_cov_matrix_virginica)
    
print("setosa_density: ",setosa_density)
print("versicolor_density: ",versicolor_density)
print("virginica_density: ",virginica_density)

# finding the predicted class for each sample in test set
y_pred = np.zeros(X_test.shape[0], dtype=np.int8)

for i in range(X_test.shape[0]):
    if setosa_density[i] > versicolor_density[i] and setosa_density[i] > virginica_density[i]:
        y_pred[i] = 0
    elif versicolor_density[i] > setosa_density[i] and versicolor_density[i] > virginica_density[i]:
        y_pred[i] = 1
    else:
        y_pred[i] = 2
    
print("y-predicted matrix: ",y_pred)


setosa_density:  [1.24407503e-005 2.88331391e-009 3.62930428e-006 8.91470361e-009
 1.59690416e-007 4.94838660e-006 4.81870896e-006 1.42994152e-005
 1.76132097e-005 2.56759711e-005 2.00656081e-081 3.74741333e-094
 9.84188481e-068 1.93191258e-043 3.10303228e-077 1.65016219e-071
 7.84946428e-076 7.49743430e-080 6.77599964e-040 3.86943752e-073
 3.50751631e-203 3.53603050e-173 1.42816582e-145 1.25535078e-208
 7.47021929e-215 6.44258088e-178 6.50503148e-144 1.09707066e-153
 4.42345505e-183 1.75408108e-136]
versicolor_density:  [1.35233877e-27 3.98494296e-17 3.91910314e-24 8.23297573e-23
 1.45100189e-25 4.62548334e-21 1.05095164e-30 1.33037935e-23
 2.58270207e-30 2.12391567e-25 5.52855844e-06 5.61481699e-05
 9.01386940e-05 8.43809835e-06 7.64849755e-05 1.12086003e-05
 6.22478428e-05 8.67483106e-05 2.11176768e-07 9.13925124e-05
 3.79760714e-13 2.52440348e-14 4.70628639e-08 2.31886271e-10
 6.69158974e-14 1.46663394e-13 5.40819455e-09 1.11299987e-07
 1.45919556e-10 1.68277728e-06]
virginica_dens

## accuracy of your classifier on the testing set and report the results.

In [34]:
print("y-predicted matrix\n",y_pred)
print("y-test matrix\n",y_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
print("percent accuracy score : ",accuracy_score(y_test, y_pred)*100)

y-predicted matrix
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2]
y-test matrix
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2]
[[10  0  0]
 [ 0 10  0]
 [ 0  0 10]]
percent accuracy score :  100.0


In [55]:
# xshape print
print(X_test.shape[0]//3)
print(setosa_density[0])

10
1.2440750254651893e-05


## disriminant function

In [56]:
apriori_prob = 1/3
g_setosa = np.zeros(10)
g_versicolor = np.zeros(10)
g_virginica = np.zeros(10)

#calculation of discriminant function
def discriminant_function(density, apriori_prob):
    return np.log(density) + np.log(apriori_prob)
    
for i in range(10):
    
    g_setosa[i] = discriminant_function(setosa_density[i], apriori_prob)
    g_versicolor[i] = discriminant_function(versicolor_density[i], apriori_prob)
    g_virginica[i] = discriminant_function(virginica_density[i], apriori_prob)

print("g_setosa: ",g_setosa)
print("g_versicolor: ",g_versicolor)
print("g_virginica: ",g_virginica)




g_setosa:  [-12.39314545 -20.76293783 -13.62508187 -19.63417612 -16.74864108
 -13.31506126 -13.34161681 -12.2539042  -12.04547368 -11.66856727]
g_versicolor:  [-62.96657429 -38.86003587 -54.99479168 -51.94992191 -58.29098534
 -47.92131837 -70.126469   -53.7726053  -69.22732891 -57.90997821]
g_virginica:  [-90.60987293 -68.25270662 -76.06780395 -74.83334161 -76.90236905
 -75.76673338 -90.69961773 -76.46688143 -95.97374117 -85.87484408]
